<a href="https://colab.research.google.com/github/Stephensingletary/Image-Classification-For-Cross-Analysis-of-Chest-X-Ray/blob/Patrick/model6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.densenet import DenseNet121

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D,Flatten,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential

from tensorflow.keras.models import load_model
from glob import glob
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report

# Create the variable `count` as a Pandas DataFrame
count = pd.DataFrame()




In [28]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [45]:
import pandas as pd

# Example DataFrame with 112,000 images
# Replace this with actual data loading code
df = pd.DataFrame({
    'Image Index': [f'image_{i}.png' for i in range(112000)]
})
# Example dictionary mapping image indices to paths
full_img_paths = {f'image_{i}.png': f'/content/drive/MyDrive/xray_images_full_dataset/images/{i}.png' for i in range(112000)}

# Check if we are dealing with a copy
if df._is_view:
    df = df.copy()

# Safely update the DataFrame using .loc
df.loc[:, 'full_path'] = df['Image Index'].map(full_img_paths.get)

# Display a small sample to verify
print(df.head())


   Image Index                                          full_path
0  image_0.png  /content/drive/MyDrive/xray_images_full_datase...
1  image_1.png  /content/drive/MyDrive/xray_images_full_datase...
2  image_2.png  /content/drive/MyDrive/xray_images_full_datase...
3  image_3.png  /content/drive/MyDrive/xray_images_full_datase...
4  image_4.png  /content/drive/MyDrive/xray_images_full_datase...


In [43]:
def CreateBalancedDataframe(_df, samples_n=8000):
    # Check if the 'target' column exists
    if 'target' not in _df.columns:
        raise ValueError("DataFrame does not have a 'target' column.")

    # Ensure we are working with copies to avoid SettingWithCopyWarning
    target_y = _df[_df.target == 1].copy().reset_index(drop=True).iloc[:samples_n]
    target_n = _df[_df.target == 0].copy().reset_index(drop=True).iloc[:samples_n]

    # Concatenate the balanced data
    balanced_df = pd.concat([target_y, target_n]).reset_index(drop=True)

    return balanced_df


In [46]:
df.head()

,Image Index,full_path
0,image_0.png,/content/drive/MyDrive/xray_images_full_datase...
1,image_1.png,/content/drive/MyDrive/xray_images_full_datase...
2,image_2.png,/content/drive/MyDrive/xray_images_full_datase...
3,image_3.png,/content/drive/MyDrive/xray_images_full_datase...
4,image_4.png,/content/drive/MyDrive/xray_images_full_datase...


In [48]:
train_set, test_set = train_test_split(df, test_size = 0.2)

# quick check to see that the training and test set were split properly
print('training set - # of observations: ', len(train_set))
print('test set - # of observations): ', len(test_set))
print('prior, full data set - # of observations): ', len(df))

training set - # of observations:  89600
test set - # of observations):  22400
prior, full data set - # of observations):  112000


In [51]:
# Check if the full_img_paths dictionary is empty
if not full_img_paths:
  raise ValueError("The full_img_paths dictionary is empty. Please ensure that it is correctly populated before proceeding.")

In [52]:
# Check if df is a copy
if df._is_view:
  raise ValueError("The df DataFrame is a view. Please ensure that you are working with a copy of the DataFrame before proceeding.")

In [60]:
# Check if the 'full_path' column exists
if 'full_path' not in df.columns:
  raise ValueError("The 'full_path' column was not successfully added to the df DataFrame. Please ensure that the code in 'ipython-input-45-63bef442b434' runs successfully.")

ValueError: The 'full_path' column was not successfully added to the df DataFrame. Please ensure that the code in 'ipython-input-45-63bef442b434' runs successfully.

In [56]:
df = df.copy()

In [58]:
df.head()

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11
0,00000001_000.png,Cardiomegaly,0,1,58,M,PA,2682,2749,0.143,0.143,NaN
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,0.143,NaN
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,0.168,NaN
3,00000002_000.png,No Finding,0,2,81,M,PA,2500,2048,0.171,0.171,NaN
4,00000003_000.png,Hernia,0,3,81,F,PA,2582,2991,0.143,0.143,NaN


In [59]:
# Check if the 'full_path' column exists
if 'full_path' not in df.columns:
  raise ValueError("The 'full_path' column was not successfully added to the df DataFrame. Please ensure that the code in 'ipython-input-45-63bef442b434' runs successfully.")

ValueError: The 'full_path' column was not successfully added to the df DataFrame. Please ensure that the code in 'ipython-input-45-63bef442b434' runs successfully.

In [50]:
# Load the data from the original source with the target column included
df = pd.read_csv('/content/drive/MyDrive/xray_images_full_dataset/Data_Entry_2017.csv')

# Split the data into training and test sets
train_set, test_set = train_test_split(df, test_size = 0.2)

# Create the data generators
data_gen = ImageDataGenerator(rescale=1/255.0,samplewise_center=True,samplewise_std_normalization= True)

train_gen = data_gen.flow_from_dataframe(train_set,batch_size=128,x_col="full_path",y_col="target",target_size=(224,224),class_mode="binary",color_mode="rgb",shuffle=True)
test_gen = data_gen.flow_from_dataframe(test_set,batch_size=128,x_col="full_path",y_col="target",target_size=(224,224),class_mode="binary",color_mode="rgb",shuffle=True)

KeyError: 'full_path'

In [24]:
import tensorflow as tf

In [25]:
def build_model():
    model_1 = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    for layer in model_1.layers:
        layer.trainable = False

    model_2 = DenseNet121(include_top=False,input_shape=(224,224,3))
    for layer in model_2.layers:
        layer.trainable=False

    input_layer = tf.keras.layers.Input(shape=(224,224, 3))
    out1 = model_1(input_layer)
    out2 = model_2(input_layer)

    fc1 = Flatten()(out1)
    fc2 = Flatten()(out2)
    fc_concat = tf.keras.layers.Concatenate()([fc1, fc2])
    fc3 = Dense(1024,activation="relu")(fc_concat)
    drop = Dropout(0.45)(fc3)
    fc_final = Dense(512,activation="relu")(drop)

    out = Dense(1,activation="sigmoid")(fc_final)
    return Model(inputs=input_layer,outputs=out)




hybrid_model = build_model()
loss = tf.keras.losses.BinaryCrossentropy()
hybrid_model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])




# model.compile(optimizer='rmsprop',
#                   loss=loss, #'binary_crossentropy'
#                   metrics=['accuracy'])

In [26]:
hybrid_model.fit(train_gen,validation_data=test_gen,epochs=10,verbose=1,steps_per_epoch=32)

NameError: name 'train_gen' is not defined

In [ ]:
hybrid_model.evaluate(test_gen)

25/25 ━━━━━━━━━━━━━━━━━━━━ 55s 2s/step - accuracy: 0.6480 - loss: 0.6371


[0.6322828531265259, 0.6575000286102295]

In [39]:
hybrid_model.save("last_model.keras")

In [41]:
# Import the ImageDataGenerator library
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the test data generator
test_gen = ImageDataGenerator()

# Set the directory containing the test data
test_gen.flow_from_directory(
    directory='path/to/test/data',
    target_size=(224, 224),
    batch_size=32
)

# Predict on the test data
predictions = hybrid_model.predict(test_gen)

FileNotFoundError: [Errno 2] No such file or directory: 'path/to/test/data'

In [40]:
 predictions = hybrid_model.predict(test_gen)
# print(predictions)
# y_classes = predictions.argmax(axis=-1)
# print(y_classes)

NameError: name 'test_gen' is not defined

In [ ]:
valid_gen = data_gen.flow_from_dataframe(test_set,batch_size=3200,x_col="full_path",y_col="target",target_size=(224,224),class_mode="binary",color_mode="rgb",shuffle=True)

Found 3200 validated image filenames belonging to 2 classes.


In [ ]:
X_test.shape

(1, 224, 224, 3)

In [ ]:
X_test, y_test = next(valid_gen)

#y_predictions = hybrid_model.predict(X_test)

In [ ]:
y_predictions

array([[0.4563325]], dtype=float32)

In [ ]:
y_classes = []
for p in y_predictions:
    if p > 0.5:
        y_classes.append(1)
    else:
        y_classes.append(0)

In [ ]:
accuracy = accuracy_score(y_test,y_classes)
print(accuracy)

1.0


In [ ]:
print(classification_report(y_test,y_classes))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00         1

    accuracy                           1.00         1
   macro avg       1.00      1.00      1.00         1
weighted avg       1.00      1.00      1.00         1



In [ ]:
accuracy = accuracy_score(ground_truth,y_classes)
print(accuracy)

In [ ]:
print(classification_report(ground_truth,y_classes))

In [ ]:
accuracy

In [ ]:
test_gen.classes

In [ ]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in base_model.layers:
    layer.trainable = False
model =Sequential([

base_model,
Flatten(),
Dense(1,activation="sigmoid")
])

# compile model, run summary
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:

model.fit(train_gen,validation_data=test_gen,epochs=10,verbose=1,steps_per_epoch=32)

In [ ]:
model.evaluate(test_gen)

In [ ]:
predictions = model.predict(test_gen)

In [ ]:
from sklearn.metrics import accuracy_score,classification_report

In [ ]:
predictions[0]

In [ ]:
np.where(predictions.ravel() < 0.5)

In [ ]:
np.unique(pred)